# 🔍 Analyse Exploratoire des Données de Fraude Bancaire

Ce notebook présente une analyse détaillée des données de transactions bancaires pour la détection de fraudes. Nous allons explorer les caractéristiques des transactions frauduleuses et légitimes, identifier des patterns et préparer les données pour le machine learning.

## 📋 Table des Matières

1. [Configuration et Imports](#setup)
2. [Chargement et Aperçu des Données](#loading)
3. [Analyse du Déséquilibre des Classes](#imbalance)
4. [Distribution des Montants](#amounts)
5. [Analyse Temporelle](#temporal)
6. [Analyse des Features Anonymisées](#features)
7. [Corrélations et Relations](#correlations)
8. [Tests Statistiques](#stats)
9. [Conclusions et Recommandations](#conclusions)

## 1. Configuration et Imports {#setup}

Importation des bibliothèques nécessaires et configuration de l'environnement.

In [ ]:
import sys
from pathlib import Path

# Ajout du répertoire source au path Python
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Imports standards
import pandas as pd
import numpy as np
from scipy import stats
import yaml

# Visualisation
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Import du loader personnalisé
from src.data.data_loader import DataLoader

# Configuration de l'affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## 2. Chargement et Aperçu des Données {#loading}

In [ ]:
# Chargement des données
loader = DataLoader("../config/config.yaml")
data = loader.load_data()

print("Dimensions du dataset :", data.shape)
data.head()

In [ ]:
# Informations sur le dataset
print("\nInformations sur le dataset :")
data.info()

print("\nStatistiques descriptives :")
data.describe()

## 3. Analyse du Déséquilibre des Classes {#imbalance}

In [ ]:
# Calcul des proportions de fraude
class_counts = data['Class'].value_counts()
class_proportions = data['Class'].value_counts(normalize=True)

# Création du graphique camembert interactif
labels = ['Transactions Légitimes', 'Transactions Frauduleuses']
values = class_counts.values

fig = go.Figure(data=[go.Pie(
    labels=labels,
    values=values,
    hole=0.4,
    textinfo='label+percent',
    marker=dict(colors=['#2ecc71', '#e74c3c'])
)])

fig.update_layout(
    title='Distribution des Classes',
    annotations=[dict(text='Déséquilibre', x=0.5, y=0.5, font_size=20, showarrow=False)]
)

fig.show()

print(f"\nNombre de transactions légitimes : {class_counts[0]:,}")
print(f"Nombre de transactions frauduleuses : {class_counts[1]:,}")
print(f"Ratio de déséquilibre : 1:{class_counts[0]/class_counts[1]:.1f}")

## 4. Distribution des Montants {#amounts}

In [ ]:
# Création d'un graphique de distribution des montants par classe
fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=('Distribution des Montants - Échelle Normale',
                                   'Distribution des Montants - Échelle Log'))

# Transactions légitimes
fig.add_trace(
    go.Histogram(x=data[data['Class']==0]['Amount'],
                 name='Légitimes',
                 marker_color='#2ecc71',
                 opacity=0.75),
    row=1, col=1
)

# Transactions frauduleuses
fig.add_trace(
    go.Histogram(x=data[data['Class']==1]['Amount'],
                 name='Frauduleuses',
                 marker_color='#e74c3c',
                 opacity=0.75),
    row=1, col=1
)

# Même chose en échelle log
fig.add_trace(
    go.Histogram(x=data[data['Class']==0]['Amount'],
                 name='Légitimes',
                 marker_color='#2ecc71',
                 opacity=0.75),
    row=2, col=1
)

fig.add_trace(
    go.Histogram(x=data[data['Class']==1]['Amount'],
                 name='Frauduleuses',
                 marker_color='#e74c3c',
                 opacity=0.75),
    row=2, col=1
)

fig.update_xaxes(title_text="Montant", row=1, col=1)
fig.update_xaxes(title_text="Montant (log)", type="log", row=2, col=1)
fig.update_yaxes(title_text="Nombre de transactions", row=1, col=1)
fig.update_yaxes(title_text="Nombre de transactions", row=2, col=1)

fig.update_layout(
    height=800,
    title_text="Distribution des Montants par Classe",
    showlegend=True
)

fig.show()

# Statistiques descriptives par classe
print("\nStatistiques des montants par classe :")
print(data.groupby('Class')['Amount'].describe())

## 5. Analyse Temporelle {#temporal}

In [ ]:
# Conversion du temps en heures
data['Hour'] = data['Time'] / 3600

# Création d'un graphique de répartition temporelle des fraudes
fig = go.Figure()

# Ajout des transactions légitimes
fig.add_trace(go.Scatter(
    x=data[data['Class']==0]['Hour'],
    y=data[data['Class']==0]['Amount'],
    mode='markers',
    name='Légitimes',
    marker=dict(color='#2ecc71', size=5, opacity=0.5)
))

# Ajout des transactions frauduleuses
fig.add_trace(go.Scatter(
    x=data[data['Class']==1]['Hour'],
    y=data[data['Class']==1]['Amount'],
    mode='markers',
    name='Frauduleuses',
    marker=dict(color='#e74c3c', size=8)
))

fig.update_layout(
    title='Distribution Temporelle des Transactions',
    xaxis_title='Temps (heures)',
    yaxis_title='Montant',
    height=600
)

fig.show()

# Calcul du taux de fraude par heure
hourly_fraud_rate = data.groupby(
    pd.cut(data['Hour'], bins=24)
)['Class'].mean()

print("\nTaux de fraude par tranche horaire :")
print(hourly_fraud_rate)

## 6. Analyse des Features Anonymisées {#features}

In [ ]:
# Création d'une fonction pour analyser la distribution d'une feature
def plot_feature_distribution(data, feature, title=None):
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=('Distribution par Classe', 'Box Plot par Classe'))
    
    # KDE plot
    for class_value in [0, 1]:
        fig.add_trace(
            go.Histogram(
                x=data[data['Class']==class_value][feature],
                name=f"{'Frauduleuse' if class_value==1 else 'Légitime'}",
                marker_color='#e74c3c' if class_value==1 else '#2ecc71',
                opacity=0.7
            ),
            row=1, col=1
        )
    
    # Box plot
    fig.add_trace(
        go.Box(
            y=data[data['Class']==0][feature],
            name='Légitime',
            marker_color='#2ecc71'
        ),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Box(
            y=data[data['Class']==1][feature],
            name='Frauduleuse',
            marker_color='#e74c3c'
        ),
        row=1, col=2
    )
    
    fig.update_layout(
        height=400,
        title_text=title if title else f"Distribution de {feature}"
    )
    
    return fig

# Analyse des premières features V1-V4
for feature in ['V1', 'V2', 'V3', 'V4']:
    fig = plot_feature_distribution(data, feature)
    fig.show()

## 7. Corrélations et Relations {#correlations}

In [ ]:
# Calcul de la matrice de corrélation
correlation_matrix = data.corr()

# Création de la heatmap de corrélation
fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='RdBu',
    zmin=-1,
    zmax=1
))

fig.update_layout(
    title='Matrice de Corrélation',
    height=800,
    width=800
)

fig.show()

# Identification des corrélations importantes avec la classe
correlations_with_class = correlation_matrix['Class'].sort_values(ascending=False)
print("\nTop 10 des features les plus corrélées avec la fraude :")
print(correlations_with_class[1:11])  # Exclusion de la corrélation avec elle-même

## 8. Tests Statistiques {#stats}

In [ ]:
def perform_statistical_tests(data, feature):
    # Séparation des données par classe
    legitimate = data[data['Class']==0][feature]
    fraudulent = data[data['Class']==1][feature]
    
    # Test de Kolmogorov-Smirnov
    ks_stat, ks_pval = stats.ks_2samp(legitimate, fraudulent)
    
    # Test de Mann-Whitney U
    mw_stat, mw_pval = stats.mannwhitneyu(legitimate, fraudulent, alternative='two-sided')
    
    return {
        'KS_statistic': ks_stat,
        'KS_p_value': ks_pval,
        'MW_statistic': mw_stat,
        'MW_p_value': mw_pval
    }

# Test pour le montant et quelques features importantes
features_to_test = ['Amount'] + [f'V{i}' for i in range(1, 5)]
test_results = {}

for feature in features_to_test:
    test_results[feature] = perform_statistical_tests(data, feature)

# Affichage des résultats
results_df = pd.DataFrame(test_results).T
print("Résultats des tests statistiques :")
print(results_df)

# Interprétation des résultats
significant_features = results_df[results_df['KS_p_value'] < 0.05].index
print("\nFeatures montrant des différences significatives entre classes :")
print(significant_features.tolist())

## 9. Conclusions et Recommandations {#conclusions}

### Principales Observations

1. **Déséquilibre des Classes**
   - Ratio très déséquilibré nécessitant des techniques spécifiques
   - Importance d'utiliser des métriques adaptées

2. **Distribution des Montants**
   - Patterns distinctifs entre transactions légitimes et frauduleuses
   - Nécessité de normalisation/standardisation

3. **Patterns Temporels**
   - Variations du taux de fraude selon l'heure
   - Opportunité pour des features temporelles

4. **Features Discriminantes**
   - Identification des variables les plus importantes
   - Base pour la sélection de features

### Recommandations pour la Suite

1. **Preprocessing**
   - Standardisation des features numériques
   - Gestion des outliers identifiés

2. **Feature Engineering**
   - Création de features temporelles
   - Agrégations par fenêtres glissantes

3. **Modélisation**
   - Utilisation de techniques de resampling
   - Focus sur les métriques adaptées au déséquilibre

4. **Évaluation**
   - Validation croisée stratifiée
   - Attention particulière aux faux positifs/négatifs